# Itinerary Generation

The purpose of this notebook is to generate the course site's itinerary page and subpages based on `Schedule.ipynb`

In [19]:
from datetime import date, timedelta
from helpers import calendar_table

In [20]:
SITE_DIR="../../../site"

In [21]:
%reload_ext autoreload
%run Schedule.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Date,Weekday,Plan
Week 0
May-15,Wed,FDOC
,,Day 00
May-16,Thu,Drop Period Ends
,,Widthdraw Period Ends
,,Day 01
May-17,Fri,Day 02
Week 1
May-20,Mon,Day 03
May-21,Tue,Day 04
May-22,Wed,Day 05
May-23,Thu,Day 06
May-24,Fri,Day 07
Week 2
May-27,Mon,University Holiday
May-28,Tue,Day 08
May-29,Wed,Day 09
May-30,Thu,Day 10
May-31,Fri,Day 11
Week 3
Jun-03,Mon,Day 12
Jun-04,Tue,Day 13
Jun-05,Wed,Day 14
Jun-06,Thu,Day 15
Jun-07,Fri,Day 16
Week 4
Jun-10,Mon,Day 17
Jun-11,Tue,Day 18
Jun-12,Wed,Day 19
Jun-13,Thu,Day 20
Jun-14,Fri,Day 21
Week 5
Jun-17,Mon,Day 22
Jun-18,Tue,LDOC
,,Day 23
Jun-21,Fri,Final @ 8:00AM


We're going to curate what is in the full syllabus based on what we want to expose to students.

In [22]:
today = date.today() - timedelta(2)  # Hack to keep the past three days in range for now, need to rewrite for deadlines

def keep(plan):
    # Don't show "Class" or "Milestone"
    if plan.kind == Kind.MS or plan.kind == Kind.DK or plan.kind == Kind.NA:
        return False
    elif plan.kind == Kind.WD and plan.date - today > timedelta(6):
        # Filter holidays out in the far future, but keep nearby ones in for
        # positive vibes and unrelenting optimism.
        return False
    elif (plan.kind == Kind.CL or plan.kind == Kind.RV) and len(plan.links) == 0:
        return False
    elif plan.kind != Kind.QZ and plan.kind != Kind.EX and plan.kind != Kind.RD and plan.kind != Kind.FN and plan.date - today > timedelta(6):
        return False
    else:
        return True

for day in syllabus:
    remove = []
    syllabus[day] = [plan for plan in syllabus[day] if keep(plan)]

show_calendar(syllabus)

Date,Weekday,Plan
Week 0,,
May-15,Wed,Welcome to COMP110 - An Intro to the Class
,,Out: Syllabus and Support
,,Out: Hello World
,,Due: Syllabus and Support
May-16,Thu,An Introduction to Coding + Beginning Concepts
,,Out: Objects and Data Types
,,Out: Expressions
,,Out: Variables and User Input
,,Due: Hello World


In [23]:
# Segment out future vs past
future = {}
past = {}
for day in syllabus:
    if day < today:
        past[day] = syllabus[day]
    else:
        future[day] = syllabus[day]

Similar to itinerary generation, we will auto-generate the 'On the Horizon' box. First, let's curate what is in the box:

In [24]:
from helpers import horizonize

horizon_syl = horizonize(syllabus_copy)
horizon_syl = [plan for plan in horizon_syl if plan.kind in {Kind.EX, Kind.QZ, Kind.RD, Kind.HW, Kind.FN, Kind.LS, Kind.PJ, Kind.CQ, Kind.VL}]

Stripping data to only near future quizzes and due dates

In [25]:
horizoninfo = []
today = date.today()

for plan in horizon_syl:
    if plan.date > (today - timedelta(1)) and plan.date < (today + timedelta(8)):
        horizoninfo.append(plan)

print(horizoninfo)

[<helpers.HorizonPlan object at 0x11a74ac50>, <helpers.HorizonPlan object at 0x11a7d8cd0>, <helpers.HorizonPlan object at 0x11a7daf90>, <helpers.HorizonPlan object at 0x11a7daa10>, <helpers.HorizonPlan object at 0x11e8d9690>]


Working to add deadline in titles of assignments where assignment date != due date

In [26]:
import calendar
syllabus: dict[date, list[Plan]]
for day in syllabus:
    for plan in syllabus[day]: 
        if plan.deadline != plan.extension:
            newdate = plan.extension.strftime("%-m/%-d")
            date = plan.deadline.strftime("%-m/%-d")
            plan.title += f"<s><i>(Due {calendar.day_name[plan.deadline.weekday()]} {date})</i></s>" 
            plan.title += f" (Extended to {calendar.day_name[plan.extension.weekday()]} {newdate})" 
        elif (plan.deadline != None) and (plan.kind != Kind.QZ): 
            date = plan.deadline.strftime("%-m/%-d")
            plan.title += f" (Due {calendar.day_name[plan.deadline.weekday()]} {date})" 

And now (finally), we generate the itinerary and horizon box.

In [27]:
from jinja2 import Environment, FileSystemLoader, select_autoescape
from helpers import serialize, serialize_plan
import re
import json 
from datetime import date
env = Environment(
    loader=FileSystemLoader(SITE_DIR),
    autoescape=select_autoescape(["html", "xml"]),
    trim_blocks=True,
    lstrip_blocks=True
)
template = env.get_template("./index.jinja2")
with open(f"{SITE_DIR}/index.md", "w", encoding="utf8") as target:
    result = template.render(syllabus=syllabus, horizon_syl=horizon_syl, future=future, horizoninfo=horizoninfo, past=past, cache=date.today())
    # We strip the double spaces because pandoc can't handle HTML well :(
    stripped = re.sub(r' {2,}', '', result)
    target.write(stripped)
serializable_syllabus = serialize(syllabus) 
future_syllabus = serialize(future)  
past_syllabus = serialize(past)   
json_syllabus = json.dumps(serializable_syllabus, indent = 4)   
json_future = json.dumps(future_syllabus, indent = 4)
json_past = json.dumps(past_syllabus, indent = 4)
f = open(f"{SITE_DIR}/static/components/Itinerary/data.json", "w")
f.write(json_syllabus)
f.close()
f = open(f"{SITE_DIR}/static/components/Itinerary/future.json", "w")
f.write(json_future)
f.close()
f = open(f"{SITE_DIR}/static/components/Itinerary/past.json", "w")
f.write(json_past)
f.close()